# Twitch data analysis

Dataset nodi (streamer):
- numero persone totali raggiunte
- tempo streammato
- viewer medi
- aggiornamento community (persone totali/tempo)/viewer medi
- peak viewers
- categorie streammate
- link twitter
- dati twitter

Dataset relazioni/ matrice streamer:
- percentuale
- numero assoluto

#### Setup parameters

In [1]:
freq = 2               #Sample frequency (1=5min, 2=10min, 3=15min ecc.)
min_stream_time = 3    #Minimum live time parameter
min_watch_time = 3     #Minimum threshold of time as a spectator (min_watch_time <= min_stream_time)
min_num_spect_mean = 3 #Minimum number of spectator
interval = 5           #Interval between observations

print(f'Minimum live time threshold:\t\t\t {interval * freq * min_stream_time} min')
print(f'Minimum threshold of time as a spectator:\t {interval * freq * min_watch_time} min')

Minimum live time threshold:			 30 min
Minimum threshold of time as a spectator:	 30 min


#### Import libraries

In [23]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize
from collections import Counter
from collections import defaultdict
import gzip

## Nodes dataset

### Import data

Complete list of files

In [3]:
relpath1 = '..\DataCollection\Twitch_stream_spect\Files_stream\\'
completelist = [os.path.join(path, name) for path, subdirs, files in os.walk(relpath1) for name in files]
print("Number of files:", len(completelist))
print(*completelist[:5], sep = "\n")

Number of files: 3823
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-09.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-14.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-20.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-24.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-29.json


Sampling files based on frequency

In [4]:
flist = completelist[::freq]
print("Number of files:", len(flist))
print(*flist[:5], sep = "\n")

Number of files: 1912
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-09.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-20.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-29.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-39.json
..\DataCollection\Twitch_stream_spect\Files_stream\2022-05-05_15-49.json


Import selected files

In [5]:
data_dict = {}

for file in flist:
    start = relpath1
    end = '.json'
    time = file[len(start): - len(end)]
    time_obj = datetime.strptime(time, '%Y-%m-%d_%H-%M') #extract timestamp
    
    with open(file, 'r') as f:
        data_dict[time_obj] = json.load(f)

print("Number of keys (time intervals):", len(data_dict.keys()))

Number of keys (time intervals): 1912


Example `'pasquale890': {'spect': ['anytow'], 'game_name': 'FIFA 22', 'viewer_count': 0}`


In [6]:
relpath2 = '..\DataCollection\Other_datasets\\'

bot_list = pd.read_csv(f'{relpath2}Twitch_bot_list.csv')
bot_list = bot_list['Twitch Username'].tolist()
bot_list[:10]

['turbopascai',
 'tteyyd',
 'twitchdetails',
 '0liviajadeee',
 'judgejudysiayer',
 'thelurkertv',
 'nothingbutgay',
 'streamers_on_discord',
 '0_applebadapple_0',
 'agressive_sock']

### Remove bot

Difference between viewer and spect for a time instant, can be negative and positive:

- bot excluded from viewer count
- viewer not connected in chat
- data updated at different instants

In [7]:
spect_with_bot = {}

for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    spect_with_bot[streamer] = len(data_dict[time_obj][streamer]['spect'])
    '''
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {spect_with_bot[streamer]}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    ")
    '''

In [8]:
def remove_bot(spect_with_bot, bot_set):
    return list(set(spect_with_bot) - bot_set)

In [9]:
bot_set = set(bot_list)
for time_obj in data_dict.keys():
    for streamer in data_dict[time_obj].keys():
        data_dict[time_obj][streamer]['spect'] = remove_bot(data_dict[time_obj][streamer]['spect'], bot_set)

Check again the difference between viewer and spect

In [10]:
for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {len(data_dict[time_obj][streamer]['spect'])}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    \tBot Number: {spect_with_bot[streamer] - len(data_dict[time_obj][streamer]['spect'])}\
    ")

Juventibus
    	Viewer Count: 2824    	Spect Count: 1914    	Difference: 910    	Bot Number: 11    
ocwsport
    	Viewer Count: 2575    	Spect Count: 1966    	Difference: 609    	Bot Number: 11    
pizfn
    	Viewer Count: 2115    	Spect Count: 1576    	Difference: 539    	Bot Number: 10    
Justees
    	Viewer Count: 1768    	Spect Count: 1280    	Difference: 488    	Bot Number: 10    
Xiuder_
    	Viewer Count: 1752    	Spect Count: 1185    	Difference: 567    	Bot Number: 17    
Sabaku_no_Sutoriimaa
    	Viewer Count: 1664    	Spect Count: 1172    	Difference: 492    	Bot Number: 6    
Paoloidolo
    	Viewer Count: 1319    	Spect Count: 1049    	Difference: 270    	Bot Number: 15    
NanniTwitch
    	Viewer Count: 1277    	Spect Count: 1040    	Difference: 237    	Bot Number: 15    
Everyeyeit
    	Viewer Count: 1268    	Spect Count: 102    	Difference: 1166    	Bot Number: 16    
Terenas
    	Viewer Count: 1215    	Spect Count: 674    	Difference: 541    	Bot Number: 17    
Multipl

amarneth
    	Viewer Count: 55    	Spect Count: 41    	Difference: 14    	Bot Number: 15    
enzaperniciaro
    	Viewer Count: 54    	Spect Count: 48    	Difference: 6    	Bot Number: 9    
BaldoneTV
    	Viewer Count: 53    	Spect Count: 62    	Difference: -9    	Bot Number: 27    
francogamer93
    	Viewer Count: 51    	Spect Count: 35    	Difference: 16    	Bot Number: 17    
Cap_Thor
    	Viewer Count: 49    	Spect Count: 45    	Difference: 4    	Bot Number: 31    
AlessandraTV_
    	Viewer Count: 49    	Spect Count: 27    	Difference: 22    	Bot Number: 6    
MrBalab
    	Viewer Count: 48    	Spect Count: 6    	Difference: 42    	Bot Number: 39    
Magma_s
    	Viewer Count: 47    	Spect Count: 38    	Difference: 9    	Bot Number: 9    
Demitat
    	Viewer Count: 47    	Spect Count: 28    	Difference: 19    	Bot Number: 6    
CentroSuonoSport1015
    	Viewer Count: 46    	Spect Count: 15    	Difference: 31    	Bot Number: 36    
BikeAndRaft
    	Viewer Count: 46    	Spect Count: 3

### Obtain streamer list and prop

In [11]:
streamer_repeated = []
game_name_repeated = []
viewer_count_repeated = []
spect_count_repeated = []
spect_repeated = []

for istante, value in data_dict.items():
    streamer_repeated.extend(list(value.keys()))
    for streamer, val in value.items():
        game_name_repeated.append(val['game_name'])
        viewer_count_repeated.append(val['viewer_count'])
        spect_count_repeated.append(len(val['spect']))
        spect_repeated.append(val['spect'])

len(streamer_repeated)

2585411

In [12]:
streamer_repeated = pd.DataFrame({'streamer': streamer_repeated,
                                  'game_name': game_name_repeated,
                                  'viewer_count': viewer_count_repeated,
                                  'spect_count': spect_count_repeated,
                                  'spect': spect_repeated
                                 })
streamer_repeated

,streamer,game_name,viewer_count,spect_count,spect
0,ilMasseo,Nintendo Switch Sports,3899,3014,"[kalashbaby, se4nolo, sante926, maverarts, 08l..."
1,pizfn,Fortnite,2522,1892,"[davi_bagna, lollodma, corvetto__, snoopj14, s..."
2,Xiuder_,Fortnite,2223,1504,"[corvetto__, matthew_03_, ramon_el_frio, ilmit..."
3,ocwsport,Sports,2183,1695,"[thirrok, filippo2307, peppeldpt, micheleterel..."
4,Juventibus,Sports,1939,1358,"[louielouie1895, leijlo, avejuve, daniderfel, ..."
...,...,...,...,...,...
2585406,goldmeember,Call Of Duty: Modern Warfare,0,0,[]
2585407,3nzo_ny,Call Of Duty: Modern Warfare,0,0,[]
2585408,WoZniaK_TV,League of Legends,0,0,[]
2585409,Triac92,Life is Strange: True Colors,0,0,[]


In [13]:
streamer_grouped = streamer_repeated.groupby("streamer").agg({'viewer_count': ['count', 'mean', 'max'],
                                                              'spect_count': 'mean',
                                                              'spect': 'sum',
                                                              'game_name': (lambda x: list(set(x)))
                                                             })

In [14]:
streamer_grouped.columns = ["_".join(pair) for pair in streamer_grouped.columns]
streamer_grouped['spect_sum'] = streamer_grouped['spect_sum'].apply(set)
streamer_grouped['spect_count_tot'] = streamer_grouped['spect_sum'].apply(len)
streamer_grouped['viewer_count_count'] = streamer_grouped['viewer_count_count'] * interval * freq

streamer_grouped.sort_values('viewer_count_mean', ascending = False, inplace = True)
streamer_grouped

,viewer_count_count,viewer_count_mean,viewer_count_max,spect_count_mean,spect_sum,game_name_<lambda>,spect_count_tot
streamer,,,,,,,
LyonWGFLive,1150,14172.200000,20035,9381.139130,"{stefymax_forte, alecobbe_05, alessio07_29, ja...","[Minecraft, Gartic Phone, Fortnite, Grand Thef...",61843
Tumblurr,2930,13688.044369,32436,10624.645051,"{sal_28_v, mike040497, anopeloso33, fvck_fra__...","[Football Cup 2022, Olympic Games Tokyo 2020: ...",157839
ChristianVieriOfficial,570,11185.350877,24931,7168.526316,"{giuliozo, filippo2307, jackowacko777777, ilba...",[Sports],72202
GrenBaud,1010,10261.019802,30017,7626.831683,"{luke_1701_, simo8917, francesco_0626222, male...","[Just Chatting, Music]",86913
ZanoXVII,3500,7626.851429,13907,5659.485714,"{simone93pescara, the_francy_, anthonyd1926, s...","[Call of Duty: Modern Warfare 2, Reaction, For...",172150
...,...,...,...,...,...,...,...
alanoooooooii,10,0.000000,0,0.000000,{},[Among Us],0
natsrimer,10,0.000000,0,0.000000,{},[Hay Day],0
beppe_operativo,10,0.000000,0,0.000000,{},[Call of Duty: Warzone],0


In [15]:
streamer_grouped_cut = streamer_grouped.drop(streamer_grouped[(streamer_grouped.viewer_count_count < min_stream_time)
                                                              | (streamer_grouped.spect_count_tot < min_num_spect_mean)
                                                              | (streamer_grouped.viewer_count_mean < min_num_spect_mean)
                                                             ].index)
streamer_grouped_cut

,viewer_count_count,viewer_count_mean,viewer_count_max,spect_count_mean,spect_sum,game_name_<lambda>,spect_count_tot
streamer,,,,,,,
LyonWGFLive,1150,14172.200000,20035,9381.139130,"{stefymax_forte, alecobbe_05, alessio07_29, ja...","[Minecraft, Gartic Phone, Fortnite, Grand Thef...",61843
Tumblurr,2930,13688.044369,32436,10624.645051,"{sal_28_v, mike040497, anopeloso33, fvck_fra__...","[Football Cup 2022, Olympic Games Tokyo 2020: ...",157839
ChristianVieriOfficial,570,11185.350877,24931,7168.526316,"{giuliozo, filippo2307, jackowacko777777, ilba...",[Sports],72202
GrenBaud,1010,10261.019802,30017,7626.831683,"{luke_1701_, simo8917, francesco_0626222, male...","[Just Chatting, Music]",86913
ZanoXVII,3500,7626.851429,13907,5659.485714,"{simone93pescara, the_francy_, anthonyd1926, s...","[Call of Duty: Modern Warfare 2, Reaction, For...",172150
...,...,...,...,...,...,...,...
BrusChetta87,10,3.000000,3,3.000000,"{oy2_0, tazzy1990ds, ooorphenoo}",[FIFA 22],3
capoeira802,90,3.000000,11,2.111111,"{ticlapp000, ahahasisk, deve_34, mta310, walte...",[Games + Demos],10
apollosmoka_,90,3.000000,5,2.000000,"{siggery, olivier290507, oldcreepyore0}",[Tom Clancy's Rainbow Six Siege],3


In [16]:
columns = ['streamer', 'minutes_live', 'viewer_mean', 'viewer_peak', 'spect_mean', 'spect_list', 'game_name', 'spect_tot']
columns_ordered = ['streamer', 'minutes_live', 'viewer_mean', 'viewer_peak', 'spect_mean', 'spect_tot', 'spect_list', 'game_name']

streamer_dataset = streamer_grouped_cut.copy()
streamer_dataset.reset_index(inplace = True)
streamer_dataset.columns = columns
streamer_dataset = streamer_dataset[columns_ordered]
streamer_dataset['spect_list'] = streamer_dataset['spect_list'].apply(list)
streamer_dataset

,streamer,minutes_live,viewer_mean,viewer_peak,spect_mean,spect_tot,spect_list,game_name
0,LyonWGFLive,1150,14172.200000,20035,9381.139130,61843,"[stefymax_forte, alecobbe_05, alessio07_29, ja...","[Minecraft, Gartic Phone, Fortnite, Grand Thef..."
1,Tumblurr,2930,13688.044369,32436,10624.645051,157839,"[sal_28_v, mike040497, anopeloso33, fvck_fra__...","[Football Cup 2022, Olympic Games Tokyo 2020: ..."
2,ChristianVieriOfficial,570,11185.350877,24931,7168.526316,72202,"[giuliozo, filippo2307, jackowacko777777, ilba...",[Sports]
3,GrenBaud,1010,10261.019802,30017,7626.831683,86913,"[luke_1701_, simo8917, francesco_0626222, male...","[Just Chatting, Music]"
4,ZanoXVII,3500,7626.851429,13907,5659.485714,172150,"[simone93pescara, the_francy_, anthonyd1926, s...","[Call of Duty: Modern Warfare 2, Reaction, For..."
...,...,...,...,...,...,...,...,...
9398,BrusChetta87,10,3.000000,3,3.000000,3,"[oy2_0, tazzy1990ds, ooorphenoo]",[FIFA 22]
9399,capoeira802,90,3.000000,11,2.111111,10,"[ticlapp000, ahahasisk, deve_34, mta310, walte...",[Games + Demos]
9400,apollosmoka_,90,3.000000,5,2.000000,3,"[siggery, olivier290507, oldcreepyore0]",[Tom Clancy's Rainbow Six Siege]
9401,BeaGamerO3,50,3.000000,6,0.600000,3,"[angeloscuro04, thomasxxxd, um3ka]",[Fortnite]


### Export Dataset

In [22]:
relpath3 = '..\DataCollection\Twitch_social_link\\'

streamer_dataset.to_csv("Datasets/Streamer_dataset.csv")

with open(f'{relpath3}Streamer_list.txt', 'w', encoding='utf-8') as writeFile:
    writeFile.writelines("%s\n" % channel for channel in streamer_grouped_cut.index)

## Relations dataset

In [18]:
spect_dict = streamer_grouped_cut[['spect_sum']].to_dict()['spect_sum']
len(spect_dict.keys())

9403

In [19]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Progress')

viewerOverlapDict = {}
completedStreamers = set() #Save which streamers have been processed to avoid repeating
count = 1

for key in spect_dict:
    tempList = {}
    totalLength = len(spect_dict.keys())
    logger.info(str(count) + "/" + str(totalLength)) #Print progress so I can keep track

    for comparisonKey in spect_dict: #Loop through every key again for each key in the dictionary
        if(comparisonKey != key and comparisonKey not in completedStreamers): #If its not a self comparison and the comparison hasn't already been completed
            overlapSize = len(spect_dict[key] & spect_dict[comparisonKey]) #Find the overlap size of the two streamers using set intersection
            if len(spect_dict[key]) < len(spect_dict[comparisonKey]):
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[key])*100, 2) #If the size is over 300 add {comparisonStreamer: overlap} to the dictionary
            else:
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[comparisonKey])*100, 2)
    viewerOverlapDict[key] = tempList #Add this comparison dictionary to the larger dictionary for that streamer
    completedStreamers.add(key) #Add the streamer to completed as no comparisons using this streamer need to be done anymore
    count+=1

INFO:Progress:1/9403
INFO:Progress:2/9403
INFO:Progress:3/9403
INFO:Progress:4/9403
INFO:Progress:5/9403
INFO:Progress:6/9403
INFO:Progress:7/9403
INFO:Progress:8/9403
INFO:Progress:9/9403
INFO:Progress:10/9403
INFO:Progress:11/9403
INFO:Progress:12/9403
INFO:Progress:13/9403
INFO:Progress:14/9403
INFO:Progress:15/9403
INFO:Progress:16/9403
INFO:Progress:17/9403
INFO:Progress:18/9403
INFO:Progress:19/9403
INFO:Progress:20/9403
INFO:Progress:21/9403
INFO:Progress:22/9403
INFO:Progress:23/9403
INFO:Progress:24/9403
INFO:Progress:25/9403
INFO:Progress:26/9403
INFO:Progress:27/9403
INFO:Progress:28/9403
INFO:Progress:29/9403
INFO:Progress:30/9403
INFO:Progress:31/9403
INFO:Progress:32/9403
INFO:Progress:33/9403
INFO:Progress:34/9403
INFO:Progress:35/9403
INFO:Progress:36/9403
INFO:Progress:37/9403
INFO:Progress:38/9403
INFO:Progress:39/9403
INFO:Progress:40/9403
INFO:Progress:41/9403
INFO:Progress:42/9403
INFO:Progress:43/9403
INFO:Progress:44/9403
INFO:Progress:45/9403
INFO:Progress:46/94

INFO:Progress:362/9403
INFO:Progress:363/9403
INFO:Progress:364/9403
INFO:Progress:365/9403
INFO:Progress:366/9403
INFO:Progress:367/9403
INFO:Progress:368/9403
INFO:Progress:369/9403
INFO:Progress:370/9403
INFO:Progress:371/9403
INFO:Progress:372/9403
INFO:Progress:373/9403
INFO:Progress:374/9403
INFO:Progress:375/9403
INFO:Progress:376/9403
INFO:Progress:377/9403
INFO:Progress:378/9403
INFO:Progress:379/9403
INFO:Progress:380/9403
INFO:Progress:381/9403
INFO:Progress:382/9403
INFO:Progress:383/9403
INFO:Progress:384/9403
INFO:Progress:385/9403
INFO:Progress:386/9403
INFO:Progress:387/9403
INFO:Progress:388/9403
INFO:Progress:389/9403
INFO:Progress:390/9403
INFO:Progress:391/9403
INFO:Progress:392/9403
INFO:Progress:393/9403
INFO:Progress:394/9403
INFO:Progress:395/9403
INFO:Progress:396/9403
INFO:Progress:397/9403
INFO:Progress:398/9403
INFO:Progress:399/9403
INFO:Progress:400/9403
INFO:Progress:401/9403
INFO:Progress:402/9403
INFO:Progress:403/9403
INFO:Progress:404/9403
INFO:Progre

INFO:Progress:719/9403
INFO:Progress:720/9403
INFO:Progress:721/9403
INFO:Progress:722/9403
INFO:Progress:723/9403
INFO:Progress:724/9403
INFO:Progress:725/9403
INFO:Progress:726/9403
INFO:Progress:727/9403
INFO:Progress:728/9403
INFO:Progress:729/9403
INFO:Progress:730/9403
INFO:Progress:731/9403
INFO:Progress:732/9403
INFO:Progress:733/9403
INFO:Progress:734/9403
INFO:Progress:735/9403
INFO:Progress:736/9403
INFO:Progress:737/9403
INFO:Progress:738/9403
INFO:Progress:739/9403
INFO:Progress:740/9403
INFO:Progress:741/9403
INFO:Progress:742/9403
INFO:Progress:743/9403
INFO:Progress:744/9403
INFO:Progress:745/9403
INFO:Progress:746/9403
INFO:Progress:747/9403
INFO:Progress:748/9403
INFO:Progress:749/9403
INFO:Progress:750/9403
INFO:Progress:751/9403
INFO:Progress:752/9403
INFO:Progress:753/9403
INFO:Progress:754/9403
INFO:Progress:755/9403
INFO:Progress:756/9403
INFO:Progress:757/9403
INFO:Progress:758/9403
INFO:Progress:759/9403
INFO:Progress:760/9403
INFO:Progress:761/9403
INFO:Progre

INFO:Progress:1073/9403
INFO:Progress:1074/9403
INFO:Progress:1075/9403
INFO:Progress:1076/9403
INFO:Progress:1077/9403
INFO:Progress:1078/9403
INFO:Progress:1079/9403
INFO:Progress:1080/9403
INFO:Progress:1081/9403
INFO:Progress:1082/9403
INFO:Progress:1083/9403
INFO:Progress:1084/9403
INFO:Progress:1085/9403
INFO:Progress:1086/9403
INFO:Progress:1087/9403
INFO:Progress:1088/9403
INFO:Progress:1089/9403
INFO:Progress:1090/9403
INFO:Progress:1091/9403
INFO:Progress:1092/9403
INFO:Progress:1093/9403
INFO:Progress:1094/9403
INFO:Progress:1095/9403
INFO:Progress:1096/9403
INFO:Progress:1097/9403
INFO:Progress:1098/9403
INFO:Progress:1099/9403
INFO:Progress:1100/9403
INFO:Progress:1101/9403
INFO:Progress:1102/9403
INFO:Progress:1103/9403
INFO:Progress:1104/9403
INFO:Progress:1105/9403
INFO:Progress:1106/9403
INFO:Progress:1107/9403
INFO:Progress:1108/9403
INFO:Progress:1109/9403
INFO:Progress:1110/9403
INFO:Progress:1111/9403
INFO:Progress:1112/9403
INFO:Progress:1113/9403
INFO:Progress:11

INFO:Progress:1415/9403
INFO:Progress:1416/9403
INFO:Progress:1417/9403
INFO:Progress:1418/9403
INFO:Progress:1419/9403
INFO:Progress:1420/9403
INFO:Progress:1421/9403
INFO:Progress:1422/9403
INFO:Progress:1423/9403
INFO:Progress:1424/9403
INFO:Progress:1425/9403
INFO:Progress:1426/9403
INFO:Progress:1427/9403
INFO:Progress:1428/9403
INFO:Progress:1429/9403
INFO:Progress:1430/9403
INFO:Progress:1431/9403
INFO:Progress:1432/9403
INFO:Progress:1433/9403
INFO:Progress:1434/9403
INFO:Progress:1435/9403
INFO:Progress:1436/9403
INFO:Progress:1437/9403
INFO:Progress:1438/9403
INFO:Progress:1439/9403
INFO:Progress:1440/9403
INFO:Progress:1441/9403
INFO:Progress:1442/9403
INFO:Progress:1443/9403
INFO:Progress:1444/9403
INFO:Progress:1445/9403
INFO:Progress:1446/9403
INFO:Progress:1447/9403
INFO:Progress:1448/9403
INFO:Progress:1449/9403
INFO:Progress:1450/9403
INFO:Progress:1451/9403
INFO:Progress:1452/9403
INFO:Progress:1453/9403
INFO:Progress:1454/9403
INFO:Progress:1455/9403
INFO:Progress:14

...

INFO:Progress:7913/9403
INFO:Progress:7914/9403
INFO:Progress:7915/9403
INFO:Progress:7916/9403
INFO:Progress:7917/9403
INFO:Progress:7918/9403
INFO:Progress:7919/9403
INFO:Progress:7920/9403
INFO:Progress:7921/9403
INFO:Progress:7922/9403
INFO:Progress:7923/9403
INFO:Progress:7924/9403
INFO:Progress:7925/9403
INFO:Progress:7926/9403
INFO:Progress:7927/9403
INFO:Progress:7928/9403
INFO:Progress:7929/9403
INFO:Progress:7930/9403
INFO:Progress:7931/9403
INFO:Progress:7932/9403
INFO:Progress:7933/9403
INFO:Progress:7934/9403
INFO:Progress:7935/9403
INFO:Progress:7936/9403
INFO:Progress:7937/9403
INFO:Progress:7938/9403
INFO:Progress:7939/9403
INFO:Progress:7940/9403
INFO:Progress:7941/9403
INFO:Progress:7942/9403
INFO:Progress:7943/9403
INFO:Progress:7944/9403
INFO:Progress:7945/9403
INFO:Progress:7946/9403
INFO:Progress:7947/9403
INFO:Progress:7948/9403
INFO:Progress:7949/9403
INFO:Progress:7950/9403
INFO:Progress:7951/9403
INFO:Progress:7952/9403
INFO:Progress:7953/9403
INFO:Progress:79

INFO:Progress:8255/9403
INFO:Progress:8256/9403
INFO:Progress:8257/9403
INFO:Progress:8258/9403
INFO:Progress:8259/9403
INFO:Progress:8260/9403
INFO:Progress:8261/9403
INFO:Progress:8262/9403
INFO:Progress:8263/9403
INFO:Progress:8264/9403
INFO:Progress:8265/9403
INFO:Progress:8266/9403
INFO:Progress:8267/9403
INFO:Progress:8268/9403
INFO:Progress:8269/9403
INFO:Progress:8270/9403
INFO:Progress:8271/9403
INFO:Progress:8272/9403
INFO:Progress:8273/9403
INFO:Progress:8274/9403
INFO:Progress:8275/9403
INFO:Progress:8276/9403
INFO:Progress:8277/9403
INFO:Progress:8278/9403
INFO:Progress:8279/9403
INFO:Progress:8280/9403
INFO:Progress:8281/9403
INFO:Progress:8282/9403
INFO:Progress:8283/9403
INFO:Progress:8284/9403
INFO:Progress:8285/9403
INFO:Progress:8286/9403
INFO:Progress:8287/9403
INFO:Progress:8288/9403
INFO:Progress:8289/9403
INFO:Progress:8290/9403
INFO:Progress:8291/9403
INFO:Progress:8292/9403
INFO:Progress:8293/9403
INFO:Progress:8294/9403
INFO:Progress:8295/9403
INFO:Progress:82

INFO:Progress:8597/9403
INFO:Progress:8598/9403
INFO:Progress:8599/9403
INFO:Progress:8600/9403
INFO:Progress:8601/9403
INFO:Progress:8602/9403
INFO:Progress:8603/9403
INFO:Progress:8604/9403
INFO:Progress:8605/9403
INFO:Progress:8606/9403
INFO:Progress:8607/9403
INFO:Progress:8608/9403
INFO:Progress:8609/9403
INFO:Progress:8610/9403
INFO:Progress:8611/9403
INFO:Progress:8612/9403
INFO:Progress:8613/9403
INFO:Progress:8614/9403
INFO:Progress:8615/9403
INFO:Progress:8616/9403
INFO:Progress:8617/9403
INFO:Progress:8618/9403
INFO:Progress:8619/9403
INFO:Progress:8620/9403
INFO:Progress:8621/9403
INFO:Progress:8622/9403
INFO:Progress:8623/9403
INFO:Progress:8624/9403
INFO:Progress:8625/9403
INFO:Progress:8626/9403
INFO:Progress:8627/9403
INFO:Progress:8628/9403
INFO:Progress:8629/9403
INFO:Progress:8630/9403
INFO:Progress:8631/9403
INFO:Progress:8632/9403
INFO:Progress:8633/9403
INFO:Progress:8634/9403
INFO:Progress:8635/9403
INFO:Progress:8636/9403
INFO:Progress:8637/9403
INFO:Progress:86

INFO:Progress:8939/9403
INFO:Progress:8940/9403
INFO:Progress:8941/9403
INFO:Progress:8942/9403
INFO:Progress:8943/9403
INFO:Progress:8944/9403
INFO:Progress:8945/9403
INFO:Progress:8946/9403
INFO:Progress:8947/9403
INFO:Progress:8948/9403
INFO:Progress:8949/9403
INFO:Progress:8950/9403
INFO:Progress:8951/9403
INFO:Progress:8952/9403
INFO:Progress:8953/9403
INFO:Progress:8954/9403
INFO:Progress:8955/9403
INFO:Progress:8956/9403
INFO:Progress:8957/9403
INFO:Progress:8958/9403
INFO:Progress:8959/9403
INFO:Progress:8960/9403
INFO:Progress:8961/9403
INFO:Progress:8962/9403
INFO:Progress:8963/9403
INFO:Progress:8964/9403
INFO:Progress:8965/9403
INFO:Progress:8966/9403
INFO:Progress:8967/9403
INFO:Progress:8968/9403
INFO:Progress:8969/9403
INFO:Progress:8970/9403
INFO:Progress:8971/9403
INFO:Progress:8972/9403
INFO:Progress:8973/9403
INFO:Progress:8974/9403
INFO:Progress:8975/9403
INFO:Progress:8976/9403
INFO:Progress:8977/9403
INFO:Progress:8978/9403
INFO:Progress:8979/9403
INFO:Progress:89

INFO:Progress:9281/9403
INFO:Progress:9282/9403
INFO:Progress:9283/9403
INFO:Progress:9284/9403
INFO:Progress:9285/9403
INFO:Progress:9286/9403
INFO:Progress:9287/9403
INFO:Progress:9288/9403
INFO:Progress:9289/9403
INFO:Progress:9290/9403
INFO:Progress:9291/9403
INFO:Progress:9292/9403
INFO:Progress:9293/9403
INFO:Progress:9294/9403
INFO:Progress:9295/9403
INFO:Progress:9296/9403
INFO:Progress:9297/9403
INFO:Progress:9298/9403
INFO:Progress:9299/9403
INFO:Progress:9300/9403
INFO:Progress:9301/9403
INFO:Progress:9302/9403
INFO:Progress:9303/9403
INFO:Progress:9304/9403
INFO:Progress:9305/9403
INFO:Progress:9306/9403
INFO:Progress:9307/9403
INFO:Progress:9308/9403
INFO:Progress:9309/9403
INFO:Progress:9310/9403
INFO:Progress:9311/9403
INFO:Progress:9312/9403
INFO:Progress:9313/9403
INFO:Progress:9314/9403
INFO:Progress:9315/9403
INFO:Progress:9316/9403
INFO:Progress:9317/9403
INFO:Progress:9318/9403
INFO:Progress:9319/9403
INFO:Progress:9320/9403
INFO:Progress:9321/9403
INFO:Progress:93

In [20]:
prova = 'ilGabbrone'

print(max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get))
print(viewerOverlapDict[prova][max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get)])

tudorinho
90.48


### Export Dict

In [25]:
json_str = json.dumps(viewerOverlapDict) + "\n"
json_bytes = json_str.encode('utf-8') 

with gzip.open(f'Datasets/Relations_dict.json.gz', 'w') as writefile: 
    writefile.write(json_bytes)